<a href="https://colab.research.google.com/github/iasaurav/fyers_algo/blob/main/end_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")  # This will suppress all warnings.



import pandas as pd
kite="https://api.kite.trade/instruments"
data = pd.read_csv(kite)

data['expiry'] = pd.to_datetime(data['expiry']).dt.strftime('%d-%m-%Y')
data['fyers_watchlist'] = data['segment'].map({"NFO-OPT": 'NSE', "BFO-OPT": 'BSE',"MCX-OPT": 'MCX',"NFO-FUT": 'NSE', "BFO-FUT": 'BSE',"MCX-FUT": 'MCX'})+":"+data['tradingsymbol']

data['INSTRUMENT'] = data['segment'].map({"NFO-OPT": 'OPTIDX', "BFO-OPT": 'OPTIDX',"MCX-OPT": 'OPTFUT',"NFO-FUT": 'FUTIDX', "BFO-FUT": 'FUTIDX',"MCX-FUT": 'FUTCOM'})
data['EXCH_ID'] = data['segment'].map({"NFO-OPT": 'NSE', "BFO-OPT": 'BSE',"MCX-OPT": 'MCX',"NFO-FUT": 'NSE', "BFO-FUT": 'BSE',"MCX-FUT": 'MCX'})

data['lot_size'] = data['lot_size'].fillna(0).astype(int)
data['strike'] = data['strike'].fillna(0).astype(int)




filters = [
    {"filter_names_list": ["NIFTY", "BANKNIFTY", "FINNIFTY"], "segment": "NFO-OPT"},
    {"filter_names_list": ["SENSEX", "BANKEX"], "segment": "BFO-OPT"},
    {"filter_names_list": ["CRUDEOIL", "GOLD", "SILVER"], "segment": "MCX-OPT"},
]

# Apply filters and print results
filtered_results = [
    data[(data['name'] == name) & (data['segment'] == f['segment'])].head(1)
    for f in filters for name in f["filter_names_list"]
]


combined_filtered_data = pd.concat(filtered_results)



# Function to determine the value for the 'upstox' column based on 'name'
def fyers(row):
    if row['name'] == "NIFTY":
        return "NSE:NIFTY50-INDEX"
    elif row['name'] == "BANKNIFTY":
        return "NSE:NIFTYBANK-INDEX"
    elif row['name'] == "FINNIFTY":
        return "NSE:FINNIFTY-INDEX"
    elif row['name'] == "SENSEX":
        return "BSE:SENSEX-INDEX"
    elif row['name'] == "BANKEX":
        return "BSE:BANKEX-INDEX"

    elif row['name'] == "CRUDEOIL":
        return "MCX:"+str(row['tradingsymbol'][:13])+"FUT"


    else:
        return None

# Apply the function to create a new column
combined_filtered_data['fyers'] = combined_filtered_data.apply(fyers, axis=1)









#adavnce
# Function to determine the value for the 'upstox' column based on 'name'
def watchlist(row):
    if row['name'] == "NIFTY":
        return str(row['fyers_watchlist'][:-7])
    elif row['name'] == "BANKNIFTY":
        return str(row['fyers_watchlist'][:-7])
    elif row['name'] == "FINNIFTY":
        return str(row['fyers_watchlist'][:-7])
    elif row['name'] == "SENSEX":
        return str(row['fyers_watchlist'][:-7])
    elif row['name'] == "BANKEX":
        return str(row['fyers_watchlist'][:-7])

    elif row['name'] == "CRUDEOIL":
        return str(row['fyers_watchlist'][:-6])


    else:
        return None

# Apply the function to create a new column
combined_filtered_data['watchlist'] = combined_filtered_data.apply(watchlist, axis=1)




# Function to determine the value for the 'upstox' column based on 'name'
def yahoo(row):
    if row['name'] == "NIFTY":
        return "^NSEI"
    elif row['name'] == "BANKNIFTY":
        return "^NSEBANK"
    elif row['name'] == "FINNIFTY":
        return "NIFTY_FIN_SERVICE.NS"
    elif row['name'] == "SENSEX":
        return "^BSESN"
    elif row['name'] == "BANKEX":
        return "BSE-BANK.BO"

    elif row['name'] == "CRUDEOIL":
        return ""


    else:
        return None

# Apply the function to create a new column
combined_filtered_data['yahoo'] = combined_filtered_data.apply(yahoo, axis=1)






# Select only the required columns
required_columns = ["name", "expiry",'segment',"lot_size","strike","fyers","fyers_watchlist",'watchlist',"INSTRUMENT","EXCH_ID","yahoo"]

combined_filtered_data = combined_filtered_data[required_columns]
combined_filtered_data=combined_filtered_data.head(6)
combined_filtered_data = combined_filtered_data.sort_values(by='expiry')

combined_filtered_data['zerodha_filter']=combined_filtered_data['name']+"|"+combined_filtered_data['expiry']+"|"+combined_filtered_data['segment']
combined_filtered_data['dhan_filter']=combined_filtered_data['name']+"|"+combined_filtered_data['expiry']+"|"+combined_filtered_data['INSTRUMENT']+"|"+combined_filtered_data['EXCH_ID']


print(combined_filtered_data)






#Googlesheet
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# Authenticate and connect to Google Sheets
creds = Credentials.from_service_account_file("/content/drive/MyDrive/googlesheet.json",
                                              scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
client = gspread.authorize(creds)

# Open spreadsheet and worksheet
spreadsheet = client.open("STOCKMARKET")
worksheet = spreadsheet.worksheet("api_trading")




# Assuming filtered_dhan_sorte is your DataFrame
  # Make sure to load your DataFrame here



data = combined_filtered_data.fillna("")  # Fill missing values with empty string

# Convert DataFrame to list of lists
data_list = [combined_filtered_data.columns.tolist()] + data.values.tolist()





# Clear and update worksheet
worksheet.clear()
worksheet.update(data_list)

print("Spreadsheet updated successfully!")

            name      expiry  segment  lot_size  strike                 fyers  \
5518      SENSEX  11-02-2025  BFO-OPT        20   78100      BSE:SENSEX-INDEX   
48304      NIFTY  13-02-2025  NFO-OPT        75   23600     NSE:NIFTY50-INDEX   
33415   CRUDEOIL  17-02-2025  MCX-OPT         1    8050  MCX:CRUDEOIL25FEBFUT   
4792      BANKEX  25-02-2025  BFO-OPT        30   57100      BSE:BANKEX-INDEX   
49871  BANKNIFTY  27-02-2025  NFO-OPT        30   50400   NSE:NIFTYBANK-INDEX   
61169   FINNIFTY  27-02-2025  NFO-OPT        65   23650    NSE:FINNIFTY-INDEX   

                 fyers_watchlist           watchlist INSTRUMENT EXCH_ID  \
5518      BSE:SENSEX2521178100CE     BSE:SENSEX25211     OPTIDX     BSE   
48304      NSE:NIFTY2521323600CE      NSE:NIFTY25213     OPTIDX     NSE   
33415    MCX:CRUDEOIL25FEB8050CE   MCX:CRUDEOIL25FEB     OPTFUT     MCX   
4792      BSE:BANKEX25FEB57100CE     BSE:BANKEX25FEB     OPTIDX     BSE   
49871  NSE:BANKNIFTY25FEB50400CE  NSE:BANKNIFTY25FEB     

In [ ]:
!pip install XlsxWriter
import pandas as pd

# Read and filter data
url = "https://images.dhan.co/api-data/api-scrip-master-detailed.csv"
data = pd.read_csv(url).iloc[:, :15]
columns = ["UNDERLYING_SYMBOL", "SECURITY_ID", "LOT_SIZE", "STRIKE_PRICE", "OPTION_TYPE", "SM_EXPIRY_DATE",
           "DISPLAY_NAME", "EXCH_ID", "SEGMENT", "ISIN", 'INSTRUMENT_TYPE', 'INSTRUMENT',"SERIES"]
data = data[columns]
# Convert expiry date to proper format
data['SM_EXPIRY_DATE'] = pd.to_datetime(data['SM_EXPIRY_DATE'], errors='coerce')

# Sort the 'SM_EXPIRY_DATE' column
data = data.sort_values(by='SM_EXPIRY_DATE')
# Convert back to 'dd-mm-yyyy' format for display
data['SM_EXPIRY_DATE'] = data['SM_EXPIRY_DATE'].dt.strftime('%d-%m-%Y')
data['link']=data["EXCH_ID"]+data["SEGMENT"]+ data['SECURITY_ID'].astype(str) + ":" + data['DISPLAY_NAME']

filtered_df_equity = data[
    (data["INSTRUMENT_TYPE"] == "ES") &  # Filter for instrument type 'ES'
    (data["SERIES"] == "EQ") &          # Filter for series 'EQ'
    (data["EXCH_ID"] == "NSE")          # Filter for exchange 'NSE'
]

print(filtered_df_equity)
filtered_df_equity.to_excel("stock.xlsx", index=False)


# Filters as a list of dictionaries
filters = [
    {'UNDERLYING_SYMBOL': 'NIFTY', 'INSTRUMENT': 'OPTIDX', 'EXCH_ID': 'NSE'},
    {'UNDERLYING_SYMBOL': 'BANKNIFTY', 'INSTRUMENT': 'OPTIDX', 'EXCH_ID': 'NSE'},
    {'UNDERLYING_SYMBOL': 'FINNIFTY', 'INSTRUMENT': 'OPTIDX', 'EXCH_ID': 'NSE'},
    {'UNDERLYING_SYMBOL': 'SENSEX', 'INSTRUMENT': 'OPTIDX', 'EXCH_ID': 'BSE'},
    {'UNDERLYING_SYMBOL': 'BANKEX', 'INSTRUMENT': 'OPTIDX', 'EXCH_ID': 'BSE'},
    {'UNDERLYING_SYMBOL': 'CRUDEOIL', 'INSTRUMENT': 'OPTFUT', 'EXCH_ID': 'MCX'},


    {'UNDERLYING_SYMBOL': 'NIFTY', 'INSTRUMENT': 'FUTIDX', 'EXCH_ID': 'NSE'},
    {'UNDERLYING_SYMBOL': 'BANKNIFTY', 'INSTRUMENT': 'FUTIDX', 'EXCH_ID': 'NSE'},
    {'UNDERLYING_SYMBOL': 'FINNIFTY', 'INSTRUMENT': 'FUTIDX', 'EXCH_ID': 'NSE'},
    {'UNDERLYING_SYMBOL': 'SENSEX', 'INSTRUMENT': 'FUTIDX', 'EXCH_ID': 'BSE'},
    {'UNDERLYING_SYMBOL': 'BANKEX', 'INSTRUMENT': 'FUTIDX', 'EXCH_ID': 'BSE'},
    {'UNDERLYING_SYMBOL': 'CRUDEOIL', 'INSTRUMENT': 'FUTCOM', 'EXCH_ID': 'MCX'}




    ]

# Applying filters
filtered_data = pd.DataFrame()

for filter_dict in filters:
    temp_data = data
    for column, value in filter_dict.items():
        temp_data = temp_data[temp_data[column] == value]
    filtered_data = pd.concat([filtered_data, temp_data])

# Resetting index after filtering
filtered_data.reset_index(drop=True, inplace=True)





# Displaying the filtered data
#print(filtered_data)

filtered_data.to_excel("future-option.xlsx", index=False)





filtered_df_future = filtered_data[filtered_data["INSTRUMENT"].isin(["FUTIDX", "FUTCOM"])]
filtered_df_future.to_excel("future.xlsx", index=False)



filtered_df_option = filtered_data[filtered_data["INSTRUMENT"].isin(["OPTIDX", "OPTFUT"])]
filtered_df_option.to_excel("option.xlsx", index=False)


print(filtered_df_option)




import pandas as pd

# Load data from the given Excel files
option_df = pd.read_excel("option.xlsx", sheet_name=None)  # Read all sheets
future_df = pd.read_excel("future.xlsx", sheet_name=None)    # Read all sheets
stock_df = pd.read_excel("stock.xlsx", sheet_name=None)    # Read all sheets
# Create a new Excel writer to save the updated data
with pd.ExcelWriter("updated_indices.xlsx", engine="xlsxwriter") as writer:
    for sheet, df in option_df.items():
        df.to_excel(writer, sheet_name=sheet, startrow=0, startcol=0, index=False)  # Save in A1:L

    for sheet, df in future_df.items():
        df.to_excel(writer, sheet_name=sheet, startrow=0, startcol=26, index=False)  # Save in N1:Z


    for sheet, df in stock_df.items():
        df.to_excel(writer, sheet_name=sheet, startrow=0, startcol=52, index=False)  # Save in N1:Z

print("Data successfully saved in 'updated_indices.xlsx'.")




#googlesheet
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# Authenticate and connect to Google Sheets
creds = Credentials.from_service_account_file("/content/drive/MyDrive/googlesheet.json",
                                              scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
client = gspread.authorize(creds)

# Open spreadsheet and worksheet
spreadsheet = client.open("STOCKMARKET")
worksheet = spreadsheet.worksheet("api")

# Read Excel data and convert to list of lists
data = pd.read_excel("updated_indices.xlsx").fillna("")  # Handle missing values
data_list = [data.columns.tolist()] + data.values.tolist()

# Clear and update worksheet
worksheet.clear()
worksheet.update(data_list)

print("Spreadsheet updated successfully!")

<ipython-input-18-a3dd69bab9f1>:6: DtypeWarning: Columns (3,10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url).iloc[:, :15]


       UNDERLYING_SYMBOL  SECURITY_ID  LOT_SIZE  STRIKE_PRICE OPTION_TYPE  \
170619              ABAN           10       1.0           NaN         NaN   
170620             ARE&M          100       1.0           NaN         NaN   
170631          EMAMIPAP        10074       1.0           NaN         NaN   
170632              FACT         1008       1.0           NaN         NaN   
170635          GODREJCP        10099       1.0           NaN         NaN   
...                  ...          ...       ...           ...         ...   
178169             AXITA         9902       1.0           NaN         NaN   
178176            NEOGEN         9917       1.0           NaN         NaN   
178177           ALANKIT         9921       1.0           NaN         NaN   
178181         VIKASLIFE         9931       1.0           NaN         NaN   
178183          ONEPOINT         9939       1.0           NaN         NaN   

       SM_EXPIRY_DATE                       DISPLAY_NAME EXCH_ID SEGMENT  \

In [58]:
name = "NIFTY"
signal = "23500PE"

import warnings
warnings.filterwarnings("ignore")  # This will suppress all warnings.

import pandas as pd

kite = "https://api.kite.trade/instruments"
data = pd.read_csv(kite)
# Keep all segments for initial filtering
data = data[data['segment'].isin(['NFO-OPT', 'BFO-OPT', 'MCX-OPT'])]
data['expiry'] = pd.to_datetime(data['expiry']).dt.strftime('%d-%m-%Y')
data['lot_size'] = data['lot_size'].fillna(0).astype(int)
data['strike'] = data['strike'].fillna(0).astype(int)
data['strikep'] = data['strike'].astype(str) + data['instrument_type']
#print(data)

filters = [
    {"filter_names_list": ["NIFTY", "BANKNIFTY", "FINNIFTY"], "segment": "NFO-OPT"},
    {"filter_names_list": ["SENSEX", "BANKEX"], "segment": "BFO-OPT"},
   ]
# Apply filters and print results
filtered_results = [
    data[(data['name'] == name) & (data['segment'] == f['segment'])].head(1)
    for f in filters for name in f["filter_names_list"]
]

combined_filtered_data = pd.concat(filtered_results)
combined_filtered_data = combined_filtered_data.sort_values(by='expiry')
print(combined_filtered_data)
#//additional_code
combined_filtered_data['zerodha_filter'] = combined_filtered_data['name'] + "|" + combined_filtered_data['expiry'] + "|" + combined_filtered_data['segment']

#print(app, tab) # Assuming app and tab are defined elsewhere
combined_filtered_data = combined_filtered_data[combined_filtered_data["name"] == name]

# Check if the DataFrame is empty before accessing elements
if not combined_filtered_data.empty:
    lot_size = combined_filtered_data.iloc[0, combined_filtered_data.columns.get_loc('zerodha_filter')] # Get 'zerodha_filter' column index

    filtered_df = data[
        (data['name'] == lot_size.split("|")[0]) &
        (data['expiry'] == lot_size.split("|")[1]) &
        (data['segment'] == lot_size.split("|")[2]) & (data['strikep'] == signal)
    ]
    print(filtered_df)
else:
    print(f"No data found for {name}")

       instrument_token  exchange_token          tradingsymbol       name  \
5518          215503365          841810     SENSEX2521178100CE     SENSEX   
48304          13112578           51221      NIFTY2521323600CE      NIFTY   
4792          210649861          822851     BANKEX25FEB57100CE     BANKEX   
49871          10325250           40333  BANKNIFTY25FEB50400CE  BANKNIFTY   
61169          19477762           76085   FINNIFTY25FEB23650CE   FINNIFTY   

       last_price      expiry  strike  tick_size  lot_size instrument_type  \
5518            0  11-02-2025   78100       0.05        20              CE   
48304           0  13-02-2025   23600       0.05        75              CE   
4792            0  25-02-2025   57100       0.05        30              CE   
49871           0  27-02-2025   50400       0.05        30              CE   
61169           0  27-02-2025   23650       0.05        65              CE   

       segment exchange  strikep  
5518   BFO-OPT      BFO  78100CE 

In [ ]:
https://nsearchives.nseindia.com/content/nsccl/mpl_feb2025.csv